Required: Factor Covar matrix: nXn, Factor exposure matrix, specific_risk

In [ ]:
from cvxpy import *
import pandas as pd
import numpy as np

In [ ]:
n_stocks = 15
df_covar = pd.read_excel("Factor Covariance Matrix-0712.xlsx")

df_asset_data = pd.read_excel("US Asset Data-0712-parse.xlsx")

df_asset_data.sort_values(by=['CAPT'],ascending=False,axis=0,inplace=True)

df_specific = df_asset_data.loc[:,["TICKER","SRISL"]]

df_factor = df_asset_data.drop(['SRISL','NAME','PRICE','YLD','CAPT'],axis=1)

## Store factor exposures in numpy matrix
factor_exposure = df_factor.to_numpy()

# Scale factor exposures
X = factor_exposure[0:n_stocks,1:]/100

F = df_covar.drop(['Unnamed: 0'],axis=1).to_numpy()


factor_risk = np.dot(np.dot(X,F),X.T)

In [ ]:
### Square the specific risk to calc. variance
specific_risk = (df_specific.iloc[:n_stocks,1:]**2).to_numpy()
# Store specific risk in diagnolized matrix
specific_risk = np.diag(specific_risk.ravel())
### Risk in terms of variance
total_risk = factor_risk + specific_risk
### Get diagonal elements
diagonal_el = total_risk.diagonal()

### Problem aims to maximize specific risk


In [ ]:
import sys
# Number of variables
n = total_risk.shape[1]

# The variables vector
x = Variable(n)
orig_wt = np.ones(n)/n
cons = np.ones(n)
# The risk in xT.Q.x format
tot_risk = quad_form(x, total_risk)
risk = quad_form(x, specific_risk)
fac_risk = quad_form(x, factor_risk)
#spec_risk = quad_form(x, specific_risk)
# The core problem definition with the Problem class from CVXPY
prob = Problem(Minimize(fac_risk), [sum(x)==1, x <= 1, x>=-1,tot_risk<=400,sum_largest(x,4)<=0.35,sum(abs(x-orig_wt))<=0.75,max(specific_risk@x)<=50])

In [ ]:
#try:
prob.solve()
print ("Optimal portfolio")
print ("----------------------")
for s in range(total_risk.shape[1]):
  print (" Investment in  {}% of the portfolio".format(round(100*x.value[s],2)))
print ("----------------------")
#print ("Exp ret = {}%".format(round(100*ret.value,2)))
print ("Expected risk    = {}%".format(round(risk.value**0.5,2)))
# except:
#     e = sys.exc_info()[0]
#     print ("Error",e)

Optimal portfolio
----------------------
 Investment in  8.75% of the portfolio
 Investment in  8.75% of the portfolio
 Investment in  8.75% of the portfolio
 Investment in  8.75% of the portfolio
 Investment in  8.75% of the portfolio
 Investment in  8.75% of the portfolio
 Investment in  -18.9% of the portfolio
 Investment in  5.15% of the portfolio
 Investment in  8.75% of the portfolio
 Investment in  8.75% of the portfolio
 Investment in  8.75% of the portfolio
 Investment in  8.75% of the portfolio
 Investment in  8.75% of the portfolio
 Investment in  8.75% of the portfolio
 Investment in  8.75% of the portfolio
----------------------
Expected risk    = 5.37%
